In [ ]:
# Homework 2

In [ ]:
# Install Java (required for Spark)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and extract Apache Spark 3.5.0 with Hadoop 3
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Install required Python packages
!pip install -q findspark
!pip install pyspark==3.5.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747862 sha256=5b5de84eeb2b7db37b338b31bf37303f687ae377c3be9baf29a0ecc7d616a353
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [ ]:
import os

# Automatically detect Java version path (Java 8 or 11, whichever is installed)
java_8_path = "/usr/lib/jvm/java-8-openjdk-amd64"
java_11_path = "/usr/lib/jvm/java-11-openjdk-amd64"

# Use whichever Java version is available
if os.path.exists(java_8_path):
    os.environ["JAVA_HOME"] = java_8_path
elif os.path.exists(java_11_path):
    os.environ["JAVA_HOME"] = java_11_path
else:
    raise EnvironmentError("No compatible Java version found.")

# Set Spark path
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"


In [ ]:

# Initialize Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Customize Spark Configuration
spark = SparkSession.builder \
    .appName("Homework2_Spark_Chapter2") \
    .master("local[2]") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "1g") \
    .getOrCreate()

# Print version to confirm setup
print("Spark version:", spark.version)

# Print config settings
for item in spark.sparkContext.getConf().getAll():
    print(f"{item[0]} = {item[1]}")


Spark version: 3.5.0
spark.app.name = Homework2_Spark_Chapter2
spark.driver.port = 43905
spark.app.submitTime = 1747325727518
spark.app.id = local-1747325729910
spark.executor.id = driver
spark.sql.warehouse.dir = file:/content/spark-warehouse
spark.driver.host = a762ce3337cb
spark.driver.memory = 1g
spark.driver.extraJavaOptions = -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2010-summary.csv to 2010-summary.csv
Saving 2011-summary.csv to 2011-summary.csv
Saving 2012-summary.csv to 2012-summary.csv
Saving 2013-summary.csv to 2013-summary.csv
Saving 2014-summary.csv to 2014-summary.csv
Saving 2015-summary.csv to 2015-summary.csv


In [ ]:
# Load all the summary CSV files into one DataFrame
df = spark.read.csv("*.csv", header=True, inferSchema=True)

In [ ]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [ ]:

# Preview schema and data
df.printSchema()
df.show(5)


root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [ ]:
df.columns
df.describe().show()
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME", "count").show(5)


+-------+-----------------+-------------------+------------------+
|summary|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|             count|
+-------+-----------------+-------------------+------------------+
|  count|             1502|               1502|              1502|
|   mean|             NULL|               NULL|1718.3189081225032|
| stddev|             NULL|               NULL|22300.368619668898|
|    min|      Afghanistan|        Afghanistan|                 1|
|    max|         Zimbabwe|           Zimbabwe|            370002|
+-------+-----------------+-------------------+------------------+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+---------

In [ ]:
df.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#672 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#672 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=373]
      +- FileScan csv [DEST_COUNTRY_NAME#670,ORIGIN_COUNTRY_NAME#671,count#672] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(6 paths)[file:/content/2010-summary.csv, file:/content/2011-summary.csv, file:/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
df.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [ ]:
df.createOrReplaceTempView("df")


In [ ]:
from pyspark.sql.functions import desc

df.groupBy("DEST_COUNTRY_NAME") \
  .count() \
  .orderBy(desc("count")) \
  .show(10)


+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|    United States|  736|
|            Japan|    6|
|       Cape Verde|    6|
| French Polynesia|    6|
|          Bolivia|    6|
|         Pakistan|    6|
|          Denmark|    6|
| Marshall Islands|    6|
|            Spain|    6|
|           Panama|    6|
+-----------------+-----+
only showing top 10 rows



In [ ]:
# SQL version
sqlWay = spark.sql("""
  SELECT DEST_COUNTRY_NAME, COUNT(1)
  FROM  df
  GROUP BY DEST_COUNTRY_NAME
""")

# DataFrame API version
dataFrameWay = (
    df
      .groupBy("DEST_COUNTRY_NAME")
      .count()
)

# Show that the physical plans are identical
sqlWay.explain()
dataFrameWay.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#670, 5), ENSURE_REQUIREMENTS, [plan_id=494]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#670] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(6 paths)[file:/content/2010-summary.csv, file:/content/2011-summary.csv, file:/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#670, 5), ENSURE_REQUIREMENTS, [plan_id=507]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#670] Batched: false, DataFilters: [], Format: CSV, Locati

In [ ]:
spark.sql ("SELECT max(count) from df").take(1)

[Row(max(count)=370002)]

In [ ]:
# identical to the book’s line, followed by .take(1)
spark.sql("SELECT MAX(count) FROM df").take(1)


[Row(max(count)=370002)]

In [ ]:
from pyspark.sql.functions import max as spark_max

df.select(spark_max("count")).take(1)


[Row(max(count)=370002)]

In [ ]:
top5_sql = spark.sql("""
  SELECT DEST_COUNTRY_NAME,
         SUM(count) AS destination_total
  FROM   flight_data_2015
  GROUP  BY DEST_COUNTRY_NAME
  ORDER  BY destination_total DESC
  LIMIT  5
""")
top5_sql.show()


+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [ ]:
from pyspark.sql.functions import sum as spark_sum, desc

top5_df = (
    df.groupBy("DEST_COUNTRY_NAME")
      .agg(spark_sum("count").alias("destination_total"))
      .orderBy(desc("destination_total"))
      .limit(5)
)
top5_df.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [ ]:
top5_sql.explain()   # physical plan
top5_df.explain()    # identical physical plan


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#765L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#670,destination_total#765L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[sum(count#672)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#670, 5), ENSURE_REQUIREMENTS, [plan_id=613]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#670], functions=[partial_sum(count#672)])
            +- FileScan csv [DEST_COUNTRY_NAME#670,count#672] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(6 paths)[file:/content/2010-summary.csv, file:/content/2011-summary.csv, file:/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#788L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#670,destination_total#788L])
   +- HashAggregate(keys=

In [ ]:
df.createOrReplaceTempView("flights")

spark.sql("""
    SELECT DEST_COUNTRY_NAME, COUNT(*) as total_flights
    FROM flights
    GROUP BY DEST_COUNTRY_NAME
    ORDER BY total_flights DESC
    LIMIT 10
""").show()


+-----------------+-------------+
|DEST_COUNTRY_NAME|total_flights|
+-----------------+-------------+
|    United States|          736|
|      Philippines|            6|
|           France|            6|
|         Anguilla|            6|
|             Fiji|            6|
|         Paraguay|            6|
|           Turkey|            6|
|           Sweden|            6|
|          Germany|            6|
|           Guyana|            6|
+-----------------+-------------+



In [ ]:
from pyspark.sql.functions import desc

(
    df.groupBy("DEST_COUNTRY_NAME")
      .sum("count")
      .withColumnRenamed("sum(count)",
                         "destination_total")
      .sort(desc("destination_total"))
      .limit(5)
      .show()
)

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [ ]:
# Add a year column by file source
from pyspark.sql.functions import input_file_name, regexp_extract

df_with_year = df.withColumn("source_file", input_file_name())
df_with_year = df_with_year.withColumn("year", regexp_extract("source_file", r'(\d{4})', 1))

df_with_year.select("year", "DEST_COUNTRY_NAME", "count").show(5)


+----+-----------------+-----+
|year|DEST_COUNTRY_NAME|count|
+----+-----------------+-----+
|2010|    United States|    1|
|2010|    United States|  264|
|2010|    United States|   69|
|2010|            Egypt|   24|
|2010|Equatorial Guinea|    1|
+----+-----------------+-----+
only showing top 5 rows



In [ ]:
df_with_year.groupBy("year", "DEST_COUNTRY_NAME") \
  .sum("count") \
  .orderBy("year", desc("sum(count)")) \
  .show()
#this step has been executed to match the numbers for different years

+----+------------------+----------+
|year| DEST_COUNTRY_NAME|sum(count)|
+----+------------------+----------+
|2010|     United States|    384932|
|2010|            Canada|      8271|
|2010|            Mexico|      6200|
|2010|    United Kingdom|      1629|
|2010|           Germany|      1392|
|2010|             Japan|      1383|
|2010|Dominican Republic|      1109|
|2010|            Brazil|       995|
|2010|       The Bahamas|       903|
|2010|          Colombia|       785|
|2010|            France|       774|
|2010|           Jamaica|       733|
|2010|       South Korea|       683|
|2010|       Netherlands|       586|
|2010|       El Salvador|       519|
|2010|        Costa Rica|       477|
|2010|             China|       448|
|2010|             Spain|       422|
|2010|           Belgium|       408|
|2010|          Honduras|       391|
+----+------------------+----------+
only showing top 20 rows



In [ ]:
#end of question2

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving GlobalLandTemperatures_GlobalLandTemperaturesByCountry.csv to GlobalLandTemperatures_GlobalLandTemperaturesByCountry.csv
Saving CO2 emissions per capita per country.csv to CO2 emissions per capita per country.csv


In [ ]:
df_temp = spark.read.csv("GlobalLandTemperatures_GlobalLandTemperaturesByCountry.csv", header=True, inferSchema=True)
df_temp.printSchema()
df_temp.show(5)


root
 |-- dt: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- Country: string (nullable = true)

+----------+------------------+-----------------------------+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+------------------+-----------------------------+-------+
|1743-11-01|4.3839999999999995|                        2.294|  Åland|
|1743-12-01|              NULL|                         NULL|  Åland|
|1744-01-01|              NULL|                         NULL|  Åland|
|1744-02-01|              NULL|                         NULL|  Åland|
|1744-03-01|              NULL|                         NULL|  Åland|
+----------+------------------+-----------------------------+-------+
only showing top 5 rows



On Canvas, you will find two datasets. The first dataset contains temperature
data by countries. Date starts from 1750 for average land temperature and goes
up to 2015. Answer the following questions:

a. For which country and during what year, the highest average temperature
was observed? [5 pts]

b. Analyze the data by country over the years, and name which are the top
10 countries with the biggest change in average temperature. [5 pts]

In [ ]:
#a
from pyspark.sql.functions import year, max

df_temp = df_temp.withColumn("Year", year("dt"))
df_temp.createOrReplaceTempView("temp")

spark.sql("""
    SELECT Country, Year, AVG(AverageTemperature) as AvgTemp
    FROM temp
    GROUP BY Country, Year
    ORDER BY AvgTemp DESC
    LIMIT 1
""").show()

print("for country Djibouti in year 2013, the highest temperature was observed at 30.7447")

+--------+----+------------------+
| Country|Year|           AvgTemp|
+--------+----+------------------+
|Djibouti|2013|30.744749999999996|
+--------+----+------------------+

for country Djibouti in year 2013, the highest temperature was observed at 30.7447


In [ ]:

w_first = Window.partitionBy("Country").orderBy("Year")
w_last  = Window.partitionBy("Country").orderBy(F.desc("Year"))

first_last = (
    df_country_year
      .withColumn("FirstYear",     F.first("Year").over(w_first))
      .withColumn("FirstYearTemp", F.first("AvgTemp").over(w_first))
      .withColumn("LastYear",      F.first("Year").over(w_last))
      .withColumn("LastYearTemp",  F.first("AvgTemp").over(w_last))
      .select("Country", "FirstYear", "FirstYearTemp",
              "LastYear",  "LastYearTemp")
      .distinct()                             # one row per country
      .withColumn("TempChange",
                  F.col("LastYearTemp") - F.col("FirstYearTemp"))
)

# 3.  Top-10 warm-ups
top10_change = (
    first_last.orderBy(F.desc("TempChange"))
              .limit(10)
)
print("Top-10 countries with the largest overall rise in average land temperature:")
top10_change.show(truncate=False)


Top-10 countries with the largest overall rise in average land temperature:
+----------+---------+------------------+--------+------------------+------------------+
|Country   |FirstYear|FirstYearTemp     |LastYear|LastYearTemp      |TempChange        |
+----------+---------+------------------+--------+------------------+------------------+
|Kuwait    |1839     |12.02             |2013    |27.273375         |15.253375000000002|
|Ukraine   |1743     |1.898             |2013    |10.913499999999999|9.0155            |
|Azerbaijan|1808     |5.235             |2013    |14.173875         |8.938875          |
|Moldova   |1743     |3.4150000000000005|2013    |11.9605           |8.545499999999999 |
|Georgia   |1779     |2.3055            |2013    |10.686625         |8.381124999999999 |
|Syria     |1808     |12.096000000000002|2013    |20.021124999999998|7.925124999999996 |
|Macedonia |1743     |5.431             |2013    |13.260124999999999|7.829124999999999 |
|Romania   |1743     |3.89        

In [ ]:
#2PART

In [ ]:
co2_path = "/content/CO2 emissions per capita per country.csv"
df_co2_raw = spark.read.option("header", True).csv(co2_path)

In [ ]:
co2_raw = (
    spark.read.option("header", True)
              .option("inferSchema", True)
              .csv(co2_path)
)

years = [str(y) for y in range(1960, 2015)]               # 1960-2014
expr = (
    "stack(" + str(len(years)) + ", " +
    ", ".join([f"'{y}', `{y}`" for y in years]) +
    ") as (Year, CO2_per_capita)"
)

co2_long = (
    co2_raw.selectExpr("`Country Name` as Country", expr)
           .filter(F.col("CO2_per_capita").isNotNull())
           .withColumn("Year", F.col("Year").cast("int"))
)

co2_long.show(5, truncate=False)

+-------+----+--------------+
|Country|Year|CO2_per_capita|
+-------+----+--------------+
|Aruba  |1986|2.868319392   |
|Aruba  |1987|7.235198033   |
|Aruba  |1988|10.02617921   |
|Aruba  |1989|10.6347326    |
|Aruba  |1990|26.37450321   |
+-------+----+--------------+
only showing top 5 rows



In [ ]:
temp_60_14 = (
    df_country_year.filter((F.col("Year") >= 1960) & (F.col("Year") <= 2014))
)

merged = (
    temp_60_14.join(co2_long, on=["Country", "Year"])
              .select("Country", "Year",
                      "AvgTemp", "CO2_per_capita")
)

print(f"Rows after merge (1960-2014)  ➜  {merged.count()}")
merged.show(5, truncate=False)

Rows after merge (1960-2014)  ➜  7844
+-----------+----+------------------+--------------+
|Country    |Year|AvgTemp           |CO2_per_capita|
+-----------+----+------------------+--------------+
|Afghanistan|1971|14.823500000000001|0.166042044   |
|Afghanistan|1977|14.805416666666668|0.182963616   |
|Afghanistan|1979|14.262083333333335|0.168376671   |
|Afghanistan|1980|14.887333333333332|0.132858608   |
|Afghanistan|1984|14.245833333333332|0.234987713   |
+-----------+----+------------------+--------------+
only showing top 5 rows



In [ ]:
#PARTB
corr_val = merged.stat.corr("AvgTemp", "CO2_per_capita")
print(f"Pearson correlation (AvgTemp  vs  CO₂ per-capita)  =  {corr_val:.4f}")

Pearson correlation (AvgTemp  vs  CO₂ per-capita)  =  -0.2345


In [ ]:
#CORELATION PER COUNTRY
from pyspark.sql import functions as F

corr_by_country = (
    merged.groupBy("Country")
          .agg(F.corr("AvgTemp", "CO2_per_capita").alias("Corr"))
          .orderBy(F.desc("Corr"))
)

corr_by_country.show(5, truncate=False)  # top 5 positive correlations

+----------------------+------------------+
|Country               |Corr              |
+----------------------+------------------+
|Oman                  |0.792979699666993 |
|British Virgin Islands|0.7912382775273427|
|Dominica              |0.7690842486975374|
|Mauritius             |0.7612099341266811|
|Pakistan              |0.759414849034336 |
+----------------------+------------------+
only showing top 5 rows

